In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests

def detect_faces_from_webcam_and_send_to_server(server_url):
    # Carregar o modelo pré-treinado do OpenCV DNN (Caffe model para detecção de rosto)
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    # Capturar a webcam
    video_capture = cv2.VideoCapture(0)

    # Verificar se a webcam foi acessada corretamente
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    print("Aguardando até que um rosto seja detectado... (Pressione Ctrl+Q para sair)")

    # Variável para controlar o loop
    stop = False

    # Função para fechar a janela tkinter e liberar a câmera
    def on_close():
        nonlocal stop
        stop = True
        print("Fechando a janela e liberando a câmera...")
        
        # Liberar a câmera após fechar a janela
        if video_capture.isOpened():
            video_capture.release()
        
        # Destruir a janela tkinter
        root.destroy()

    # Criação da janela tkinter
    root = tk.Tk()
    root.title("Webcam Live Feed with Face Detection")

    # Criação de um label tkinter para exibir os frames da webcam
    label = tk.Label(root)
    label.pack()

    # Definir o atalho Ctrl+Q para interromper a captura e fechar a janela
    root.bind('<Control-q>', lambda event: on_close())

    # Fechar a janela ao clicar no botão de fechar
    root.protocol("WM_DELETE_WINDOW", on_close)

    def update_frame():
        # Verificar se o fechamento foi solicitado
        if stop:
            return

        # Capturar frame por frame da webcam
        ret, frame = video_capture.read()

        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        # Pré-processar a imagem para a detecção de rosto
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        # Percorrer as detecções
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # Filtrar detecções fracas (threshold de confiança pode ser ajustado)
            if confidence > 0.5:
                # Obter as coordenadas da caixa delimitadora para o rosto detectado
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # Adicionar à lista de localizações de rosto
                face_locations.append((startY, endX, endY, startX))

                # Desenhar a caixa delimitadora ao redor do rosto
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # Converter o frame para RGB (compatível com tkinter)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        # Atualizar o label tkinter com a nova imagem
        label.imgtk = imgtk
        label.configure(image=imgtk)

        # Verificar se algum rosto foi detectado
        if len(face_locations) > 0:
            print("Rosto detectado!")

            # Obter as codificações faciais para os rostos detectados
            face_encodings = face_recognition.face_encodings(frame, face_locations)

            # Preparar os dados para enviar ao servidor
            data_to_send = {
                'encodings': [encoding.tolist() for encoding in face_encodings]  # Converter arrays numpy para listas
            }

            # Enviar as codificações faciais para o servidor
            try:
                response = requests.post(server_url, json=data_to_send)
                print("Resposta do servidor:", response.json())
            except Exception as e:
                print(f"Erro ao enviar os dados para o servidor: {e}")

            # Pausar a captura de frames por 3 segundos
            root.after(3000, update_frame)  # Pausar por 3 segundos após detecção
        else:
            # Continuar a captura dos frames a cada 10 milissegundos
            label.after(10, update_frame)

    # Iniciar o loop de atualização dos frames
    update_frame()

    # Iniciar o loop do tkinter e garantir que o fechamento da janela libere a câmera
    root.mainloop()

# Testar a função
server_url = 'http://127.0.0.1:5000/recognize'
detect_faces_from_webcam_and_send_to_server(server_url)
